In [ ]:
import requests
import pandas as pd
from datetime import date, timedelta
import time

def get_nse_history(symbol, from_date, to_date, retries=3, timeout=30):
    """
    Fetches historical stock data from the NSE India API, including deliverable stats.
    """
    api_url = (
        "https://www.nseindia.com/api/historicalOR/generateSecurityWiseHistoricalData?"
        f"symbol={symbol}&series=EQ&type=priceVolumeDeliverable&"
        f"from={from_date}&to={to_date}"
    )

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en-US,en;q=0.9',
        'X-Requested-With': 'XMLHttpRequest'
    }

    session = requests.Session()
    
    for attempt in range(retries):
        try:
            report_page_url = f"https://www.nseindia.com/get-quotes/equity?symbol={symbol}"
            session.get(report_page_url, headers=headers, timeout=timeout)
            
            response = session.get(api_url, headers=headers, timeout=timeout)
            response.raise_for_status()

            data = response.json()
            df = pd.DataFrame(data['data'])

            # --- Data Cleaning and Formatting (UPDATED SECTION) ---
            df.rename(columns={
                'mTIMESTAMP': 'Date',
                'CH_SYMBOL': 'Symbol',
                'CH_SERIES': 'Series',
                'CH_OPENING_PRICE': 'Open',
                'CH_TRADE_HIGH_PRICE': 'High',
                'CH_TRADE_LOW_PRICE': 'Low',
                'CH_CLOSING_PRICE': 'Close',
                'CH_TOT_TRADED_QTY': 'Volume',
                'CH_TOTAL_TRADES': 'Trades',          # <-- ADDED
                'COP_DELIV_PERC': 'Deliverable_Perc'  # <-- ADDED
            }, inplace=True)
            
            # Select and reorder the columns, including the new ones
            df = df[[
                'Symbol', 'Series', 'Date', 'Open', 'High', 'Low', 'Close', 
                'Volume', 'Trades', 'Deliverable_Perc'
            ]]
            
            df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')
            df.set_index('Date', inplace=True)
            
            return df

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} for {symbol} failed: {e}")
            if attempt < retries - 1:
                time.sleep(2)
            else:
                print(f"All retries failed for {symbol}.")
    
    return pd.DataFrame()


# --- Main Execution ---
if __name__ == "__main__":
    tickers_list = ['RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'CANBK']
    to_date = date.today()
    from_date = to_date - timedelta(days=90)
    
    to_date_str = to_date.strftime('%d-%m-%Y')
    from_date_str = from_date.strftime('%d-%m-%Y')
    
    print(f"Fetching data for {len(tickers_list)} tickers from {from_date_str} to {to_date_str}...")

    all_tickers_data = []
    for i, ticker in enumerate(tickers_list):
        print(f"({i+1}/{len(tickers_list)}) Fetching data for {ticker}...")
        stock_df = get_nse_history(symbol=ticker, from_date=from_date_str, to_date=to_date_str)
        
        if not stock_df.empty:
            all_tickers_data.append(stock_df)
        
        time.sleep(1.5)

    if all_tickers_data:
        final_df = pd.concat(all_tickers_data)
        print("\n✅ Successfully fetched data with all requested columns:")
        print(final_df.to_string())
    else:
        print("\n❌ Could not fetch any data.")